In [ ]:
import torch
import numpy as np
import json

In [ ]:
path = 'dataset/'

In [ ]:
# JSONファイルを読み込む
with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 読み込んだデータを表示
print(data)
